<a href="https://colab.research.google.com/github/Prima-7272/BDS/blob/main/Dashbord_on_streamlit_BDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from PIL import Image

# Load the dataset
@st.cache_data
def load_data():
    data = pd.read_csv('Assignment-2/micro_world_139countries.csv', encoding='ISO-8859-1')
    return data

data = load_data()

# Title and description of the dashboard

st.title("🌍 Digital Financial Behaviors Dashboard")
st.markdown("""
    This interactive dashboard allows you to explore key insights regarding digital payment adoption, gender disparities, regional financial behaviors,
    and socioeconomic impacts using data from 139 countries. Use the filters on the sidebar to interact with the data and understand the various factors influencing digital finance adoption.
""")

# Sidebar for user input
st.sidebar.header("🔍 User Input Filters")


# Filters
region = st.sidebar.multiselect("Select Region", options=data['regionwb'].unique(), default=data['regionwb'].unique())
gender = st.sidebar.radio("Select Gender", options=["All", "Female", "Male"], index=0)
income_level = st.sidebar.multiselect("Select Income Level", options=sorted(data['inc_q'].dropna().unique()), default=data['inc_q'].dropna().unique())
show_data = st.sidebar.checkbox("Show Raw Data", False)

# Filter data based on user input
filtered_data = data[(data['regionwb'].isin(region)) & (data['inc_q'].isin(income_level))]
if gender == "Female":
    filtered_data = filtered_data[filtered_data['female'] == 1]
elif gender == "Male":
    filtered_data = filtered_data[filtered_data['female'] == 0]

# Show raw data based on checkbox
if show_data:
    st.subheader("Raw Data")
    st.write(filtered_data)

# Create columns for better layout
col1, col2 = st.columns(2)

# Key Insights Section
st.header("📊 Key Insights")

# 1. Digital Payment Usage by Gender
with col1:
    st.subheader("Digital Payment Usage by Gender")
    gender_payment = filtered_data.groupby('female')['anydigpayment'].mean().reset_index()
    gender_payment['Gender'] = gender_payment['female'].apply(lambda x: 'Female' if x == 1 else 'Male')

    fig = px.bar(gender_payment, x='Gender', y='anydigpayment', color='Gender',
                 color_discrete_sequence=px.colors.qualitative.Vivid,
                 title="Digital Payment Usage by Gender")
    fig.update_layout(yaxis_title="Percentage Using Digital Payments", xaxis_title="Gender")
    st.plotly_chart(fig, use_container_width=True)

# 2. Digital Payment Usage by Region
with col2:
    st.subheader("Digital Payment Usage by Region")
    region_payment_use = filtered_data.groupby('regionwb')['anydigpayment'].mean().sort_values(ascending=False).reset_index()

    fig = px.bar(region_payment_use, x='regionwb', y='anydigpayment', color='regionwb',
                 color_discrete_sequence=px.colors.qualitative.Pastel,
                 title="Digital Payment Usage by Region")
    fig.update_layout(yaxis_title="Percentage Using Digital Payments", xaxis_title="Region")
    fig.update_xaxes(tickangle=45)
    st.plotly_chart(fig, use_container_width=True)

# 3. Digital Payment Usage by Income Level
st.subheader("Digital Payment Usage by Income Level")
income_payment_use = filtered_data.groupby('inc_q')['anydigpayment'].mean().sort_values(ascending=False).reset_index()

fig = px.bar(income_payment_use, x='inc_q', y='anydigpayment', color='inc_q',
             color_discrete_sequence=px.colors.qualitative.Set3,
             title="Digital Payment Usage by Income Level")
fig.update_layout(yaxis_title="Percentage Using Digital Payments", xaxis_title="Income Level (Quantile)")
st.plotly_chart(fig, use_container_width=True)


# Conclusion and Footer
st.markdown("""
    ### Conclusion
    Use the filters to explore the data and identify the disparities in digital payment usage based on gender, region, income level, mobile ownership, and education level.
    The insights provided can help identify target areas for improving financial inclusion and promoting digital finance.

    **Developed by:** Sahana Irin Prima
    **Contact:** [LinkedIn](https://www.linkedin.com) | [Thanks)
""")
